# Download de Dados da Nuvem CAED

## 1. Imports e Configurações

In [2]:
import pandas as pd
from datetime import datetime
import requests
import openpyxl
import json
import os
from sqlalchemy import create_engine, VARCHAR

In [3]:
# ==================== CONFIGURAÇÕES ====================

# API
URL_BASE = 'https://parc.caeddigital.net/portal/classes'
HEADERS = {
    'X-Parse-Application-Id': 'portal',
    'X-Parse-Client-Key': 'js3.4.2',
    'X-Parse-Master-Key': '(daKjG_?x_xvba9',
    'X-Parse-Installation-Id': 'd35b5929-fcc4-45f4-8bed-44f219c2d533',
    'Content-Type': 'application/json',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64)'
}

# Banco de Dados
SERVIDOR = '192.168.250.8,61433'
BANCO_DE_DADOS = 'SDV_TESTE_IMPORTACAO'
TRUSTED_CONNECTION = 'yes'

# Processamento
BATCH_SIZE = 10000
COLLECTIONS = ['E_1308_ESTADO']

## 2. Download dos Dados da API

In [4]:
def obter_todos_dados_api(collection, batch_size=BATCH_SIZE):
    """
    Baixa todos os dados de uma collection da API.
    """
    url = f"{URL_BASE}/{collection}"
    
    all_results = []
    skip = 0
    total_retrieved = 0
    
    try:
        while True:
            payload = json.dumps({
                "where": {"excluido": False},
                "limit": batch_size,
                "skip": skip
            })
            
            response = requests.get(url, headers=HEADERS, data=payload)
            response.raise_for_status()
            
            data = response.json()
            results = data.get('results', [])
            
            if not results:
                break
                
            all_results.extend(results)
            total_retrieved += len(results)
            skip += batch_size
            
            print(f"Collection {collection}: Baixados {total_retrieved} registros...\n")
            
    except requests.exceptions.RequestException as e:
        print(f"Erro ao acessar a API para collection {collection}: {e}")
    
    return pd.json_normalize(all_results)

In [5]:
# Baixar dados das collections
dataframes = {col: obter_todos_dados_api(col) for col in COLLECTIONS}

# Criar variável de atalho para a collection principal
enturmacao = dataframes['E_1308_ESTADO']

# Mostrar informações
display(enturmacao.head())
print(f"\nTotal de registros: {len(enturmacao)}")

Collection E_1308_ESTADO: Baixados 18 registros...



,objectId,uuidLayout,uuid,CD_LEIAUTE_ARQUIVO,excluido,ID_RESULTADO,idEntidade,CD_REGISTRO_REPOSITORIO,CD_REGISTRO_CAED,CD_FONTE_REGISTRO,...,CD_FORMULARIO_PUBLICACAO,CD_PAIS,NM_PAIS,CD_REGIAO,NM_REGIAO,CD_ESTADO,NM_ESTADO,SG_ESTADO,executionId,persistDate
0,13081680300000016_E_1308_ESTADO,E_1308_ESTADO,E_1308_ESTADO,055,False,15245,15245,15245,13081680300000016,168,...,M1.COD.003.F,01058,BRASIL,1,NORTE,16,AMAPÁ,AP,ARQ_AD_055_X_1308_20221117180234_000000000,2022-11-17T18:02:43.660Z
1,13081680300000051_E_1308_ESTADO,E_1308_ESTADO,E_1308_ESTADO,055,False,15253,15253,15253,13081680300000051,168,...,M1.COD.003.F,01058,BRASIL,5,CENTRO-OESTE,51,MATO GROSSO,MT,ARQ_AD_055_X_1308_20221117180234_000000000,2022-11-17T18:02:43.660Z
2,13081680300000022_E_1308_ESTADO,E_1308_ESTADO,E_1308_ESTADO,055,False,15247,15247,15247,13081680300000022,168,...,M1.COD.003.F,01058,BRASIL,2,Nordeste,22,PIAUÍ,PI,ARQ_AD_055_X_1308_20221117180234_000000000,2022-11-17T18:02:43.660Z
3,13081680300000026_E_1308_ESTADO,E_1308_ESTADO,E_1308_ESTADO,055,False,15248,15248,15248,13081680300000026,168,...,M1.COD.003.F,01058,BRASIL,2,Nordeste,26,PERNAMBUCO,PE,ARQ_AD_055_X_1308_20221117180234_000000000,2022-11-17T18:02:43.660Z
4,13081680300000027_E_1308_ESTADO,E_1308_ESTADO,E_1308_ESTADO,055,False,15249,15249,15249,13081680300000027,168,...,M1.COD.003.F,01058,BRASIL,2,Nordeste,27,ALAGOAS,AL,ARQ_AD_055_X_1308_20221117180234_000000000,2022-11-17T18:02:43.660Z



Total de registros: 18


## 3. Preparação para Importação no SQL Server

In [6]:
# Criar engine de conexão
string_conexao = f'mssql+pyodbc://{SERVIDOR}/{BANCO_DE_DADOS}?driver=ODBC+Driver+17+for+SQL+Server&trusted_connection={TRUSTED_CONNECTION}'
engine = create_engine(string_conexao, fast_executemany=True)

print("Conexão com banco de dados criada")

Conexão com banco de dados criada


In [7]:
# Calcular tamanhos máximos das colunas
tamanhos_maximos = {}
tipos_de_dados = {}

for nome_collection, df in dataframes.items():
    # Calcula tamanho máximo de cada coluna
    tamanhos = df.map(lambda x: len(str(x))).max()
    tamanhos_maximos[nome_collection] = tamanhos
    
    # Define tipo VARCHAR com o tamanho calculado
    tipos = {coluna: VARCHAR(tamanho) for coluna, tamanho in tamanhos.items()}
    tipos_de_dados[nome_collection] = tipos

print("Tipos de dados calculados")

Tipos de dados calculados


## 4. Importação para SQL Server

In [ ]:
# Importar cada DataFrame para o SQL Server
for nome_collection, df in dataframes.items():
    print(f"\nImportando {nome_collection}...")
    
    df.to_sql(
        nome_collection,
        engine,
        index=False,
        if_exists='replace',  # Mude para 'replace' se quiser substituir a tabela
        dtype=tipos_de_dados[nome_collection]
    )
    
    print(f"✓ {nome_collection} importado com sucesso! ({len(df)} registros)")

print("\n" + "="*50)
print("IMPORTAÇÃO CONCLUÍDA COM SUCESSO!")
print("="*50)


Importando E_1308_ESTADO...
✓ E_1308_ESTADO importado com sucesso! (18 registros)

IMPORTAÇÃO CONCLUÍDA COM SUCESSO!
